# Détection automatique d'erreurs d'OCR

## Imports

In [6]:
pip install thefuzz 


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 4.2 MB/s eta 0:00:01
   --------------------------------- ------ 1.3/1.5 MB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.6 MB/s  0:00:00

   ---------------------------------------- 0/2 [rapidfuzz]
   ---------------------------------------- 0/2 [rapidfuzz]
   ---------------------------------------- 0/2 [rapidfuzz]
   ---------------------------------------- 0/2 [rapidfuzz]
   -------------------- ------------------- 1/2 [thefuzz]
   ---------------------------------------- 2/2 [thefuzz]

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install Levenshtein

Note: you may need to restart the kernel to use updated packages.


In [12]:
import nltk
import os
from thefuzz import fuzz
from thefuzz import process
from Levenshtein import distance

Nous allons d'abord utiliser la distance de Levenshtein, selon sa formule originale (1965)

In [4]:
distance("bulletin", "ballotins")

3

Nous allons ensuite faire appel à la librairie TheFuzz (anciennement fuzzywuzzy) qui repose sur python-Levenshtein (https://github.com/seatgeek/thefuzz).
Celle-ci permet de normaliser le résultat en calculant un ratio (pourcentage) de similarité entre deux chaînes de caractères :

In [5]:
fuzz.ratio("bulletin", "ballotins")

71

La fonction  `partial_ratio` se focalise sur la sous-chaîne (substring) la plus similaire : 

In [6]:
fuzz.partial_ratio("Grand", "Grand-Place")

100

La fonction `token_sort_ratio` ignore l'ordre des mots :

In [7]:
fuzz.token_sort_ratio("archives de la ville de Bruxelles", "de Bruxelles la ville de archives")

100

La fonction `token_set_ratio` ignore les répétitions de mots :

In [8]:
fuzz.token_set_ratio("plus jamais ça !", "plus jamais Jamais JAMAIS ça !!!")

100

Chargeons à présent le fichier `all.txt` (tronqué) : 

In [14]:
data_path = "C:/Users/aspng/ULB_TAC/tac/data/txt"
LIMIT = 50000000

all_text = ""

# Parcours des fichiers .txt
for filename in os.listdir(data_path):
    if filename.endswith(".txt"):
        filepath = os.path.join(data_path, filename)
        with open(filepath, "r", encoding="utf-8") as f:
            all_text += f.read()

# Limite à LIMIT caractères
text = all_text[:LIMIT]

tokens = nltk.wordpunct_tokenize(text)
print(f"{len(tokens)} words (tokens) found")

12662126 words (tokens) found


In [11]:
data_path = "C:/Users/aspng/ULB_TAC/tac/data/txt"
LIMIT = 50000000
text = open(data_path, "r", encoding="utf-8").read()[:LIMIT]

tokens = nltk.wordpunct_tokenize(text)
print(f"{len(tokens)} words (tokens) found")

vocab = set(tokens)
print(f"{len(vocab)} different word forms")

PermissionError: [Errno 13] Permission denied: 'C:/Users/aspng/ULB_TAC/tac/data/txt'

Nous pouvons identifier les variantes d'un mot par ordre décroissant de similarité : 

In [ ]:
word = "bruxelles"
process.extract(word, vocab, limit=15)

Les caractères accentués ne sont pas pris en compte pour le calcul du score, mais cela reste une bonne base pour récupérer les occurrences mal OCRisées d'un mot.